In [1]:
import spacy
from spacy.lang.en import English
import datasets
import os
import pandas
import numpy
import tqdm
import torch
import pickle

In [2]:
spacy.require_gpu()

True

In [3]:
model = English()
model.add_pipe('sentencizer')
print(model.max_length)
model.max_length = model.max_length * 1000
print(model.max_length)

1000000
1000000000


In [4]:
text_corpa = datasets.load_from_disk('cleaned_text')

In [5]:
batch_size = 16
rang = list(range(batch_size, text_corpa.num_rows, batch_size))
if(text_corpa.num_rows != rang[-1]):
    rang.append(text_corpa.num_rows)

In [6]:
sentence_corpa = []
i = 0
for j in tqdm.tqdm(rang):
    text = text_corpa[i: j]['cleaned_text']
    i = j
    with torch.no_grad():
        docs = model.pipe(text,batch_size = batch_size)
        [sentence_corpa.append(lis) for lis in [[str(sent) for sent in doc.sents] for doc in list(docs)]]
        del docs
        torch.cuda.empty_cache()

100%|██████████| 1117/1117 [12:52:16<00:00, 41.48s/it]       


In [7]:
len(sentence_corpa)

17868

In [8]:
with open("sentence_corpa.pkl", "wb") as f:
    pickle.dump(sentence_corpa, f)